In [2]:
import socket
import struct
from threading import Thread

In [3]:
#DOWNLOAD THREAD
def download_thread(fileName, clientInfo):
    print('Responsible for processing client DOWNLOAD files')
    
    #create UDP socket
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    
    fileNum = 0  #indicates serial number of the received file
    
    try:
        f = open(fileName, 'rb') #Opens a file for READING only in binary format.
    except:
        #ERROR
        #Opcode | ErrorCode | ErrMsg | 0
        #   2        2        string   1   (bytes)
        errorData = struct.pack('!HHHb', 5, 5, 5, fileNum) #error Opcode is 5
        
        #send msg when file doesnt exist
        s.sendto(errorData, clientInfo)
        
        exit()
    
    while True:
        #read file contents 512 bytes from local server 
        readFileData = f.read(512)
        
        #file number starts at 0 and adds 1 each time. range=[0,65535]
        # implement roll over block id
        if fileNum <65535:
            fileNum += 1
        else:
            fileNum.reset()  # reset filenum to 1
        def reset():
            fileNum = 1

            
        #DATA
        #Opcode | Block# | Data
        #   2       2       n    (bytes)
        sendData = struct.pack('!HH', 3, fileNum) + readFileData
        
        #data sent 1st time  
        s.sendto(sendData, clientInfo)
        
        #when data rcved by client is less than 516 bytes, then is completed
        if len(sendData) < 516:
            print('User' + str(clientInfo), end='')
            print(" :Download " + fileName + ' completed!')
            break
            
            
        
        #2nd time receiving data ACK
        recvData, clientInfo = s.recvfrom(1024)
        
        packetOpt = struct.unpack('!H', recvData[:2]) #Opcode
        packetNum = struct.unpack('!H', recvData[2:4]) #Block number
        
        if packetOpta[0] != 4 or packetNum[0] != fileNum: #4 is ACK Opcode
            print('File transfer error!')
            break
    
    # Close file        
    f.close()
    # Close UDP port
    s.close()
    # Exit the download thread
    exit()
    

In [4]:
#UPLOAD THREAD
def upload_thread(fileName, clientInfo):
    print('Responsible for processing client UPLOAD files')
    
    fileNum = 0
    
    f = open(fileName, 'wb') #Opens a file for WRITING only in binary
    
    #Create UDP port
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    
    #send ACK to client replying to client's WRQ
    sendDataFirst = struct.pack('!HH', 4, fileNum)
    s.sendto(sendDataFirst, clientInfo) #sent with random port
    
    while True:
        #recv DATA from client
        recvData, clientInfo = s.recvfrom(1024) #1st time client connects with my random port
        
        #unpack DATA
        packetOpt = struct.unpack('!H', recvData[:2]) #Opcode
        packetNum = struct.unpack('!H', recvData[2:4]) #Block num
        
        # 3 is DATA Opcode
        if packetOpt[0] == 3 and packetNum[0] == fileNum:
            #Save data to file
            f.write(recvData[4:])
            
            sendData = struct.pack('!HH', 4, fileNum)
            #reply client's ACK
            s.sendto(sendData, clientInfo) #2nd time connected to my port
            
            # implement roll-over block id
            if fileNum <65535:
                fileNum += 1
            else:
                fileNum.reset()  #reset to 1
            def reset():
                fileNum = 1
            
            ##If len(recvData) < 516 means the file goes to the end
            if len(recvData)<516:
                print('User' + str(clientInfo), end='')
                print(' :Upload '+ fileName + ' complete!')
                break
                
    f.close()
    s.close()
    exit()

In [5]:
# main function
def main():
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    
    # setsockopt(level,optname,value)
    s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    
    #bind localhost and port 69
    pars = ('127.0.0.1', 69)
    s.bind(pars)
    
    print("TFTP Server start successfully!")
    print("Server is running...")
    
    
    while True:    
        # Receive data from client
        recvData, clientInfo = s.recvfrom(1024)
        
        if struct.unpack('!b5sb', recvData[-7:]) == (0, b'octet', 0):
            opcode = struct.unpack('!H',recvData[:2])   #　Opcode
            fileName = recvData[2:-7].decode('ascii')  #　Filename
            
            # check opcode
            # Read Request DOWNLOAD
            if opcode[0] == 1:
                t = Thread(target=download_thread, args=(fileName, clientInfo))
                t.start()
                
            # Write Request UPLOAD
            if opcode[0] == 2:
                t = Thread(target=upload_thread, args=(fileName, clientInfo))
                t.start()
                
    s.close()

In [ ]:
# call main func
if __name__ == '__main__':
    main()

TFTP Server start successfully!
Server is running...
Responsible for processing client UPLOAD files
Responsible for processing client DOWNLOAD files
User('127.0.0.1', 62008) :Download bigfile completed!
